In [1]:
import torch

import torchvision
import torchvision.transforms as transforms
from torchvision import models

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from torch.nn import Parameter

## Load in dataset, model, etc.

In [2]:
# Load in data and such
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
# CIFAR10 images are 3x32x32, 3-channel 32x32 images

class Net(nn.Module):
    def __init__(self, train_drop1, train_drop2, train_drop3, train_drop4, train_drop5):    
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 3, padding=1)   # (in-channels, out-channels, kernel size)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(train_drop1)
        self.conv2 = nn.Conv2d(12, 32, 3, padding=1)
        self.dropout2 = nn.Dropout(train_drop2)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.dropout3 = nn.Dropout(train_drop3)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.dropout4 = nn.Dropout(train_drop4)
        self.fc2 = nn.Linear(128, 128)
        self.dropout5 = nn.Dropout(train_drop5)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x, bool, drop1, drop2, drop3, drop4, drop5):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout1(x)        
        x = F.dropout(x, drop1, bool)

        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout2(x)
        x = F.dropout(x, drop2, bool)
        
        x = self.pool(F.relu(self.conv3(x)))
        x = x.reshape(-1, 64 * 4 * 4)
        x = self.dropout3(x)
        x = F.dropout(x, drop3, bool)
        
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)
        x = F.dropout(x, drop4, bool)
        
        x = F.relu(self.fc2(x))
        x = self.dropout5(x)
        x = F.dropout(x, drop5, bool)
    
        x = self.fc3(x)
        
        return x

Files already downloaded and verified
Files already downloaded and verified


### Example loading in a network then dynamically quantizing it

In [4]:
PATH = './sample_models/cifar_net_sgd_72_41.pth'

quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
quant_net.load_state_dict(torch.load(PATH))
quant_net = torch.quantization.quantize_dynamic(quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)

print(quant_net)  # Can see here that only the linear layers have been quantized

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0, inplace=False)
  (conv2): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout2): Dropout(p=0, inplace=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout3): Dropout(p=0, inplace=False)
  (fc1): DynamicQuantizedLinear(in_features=1024, out_features=128, qscheme=torch.per_tensor_affine)
  (dropout4): Dropout(p=0, inplace=False)
  (fc2): DynamicQuantizedLinear(in_features=128, out_features=128, qscheme=torch.per_tensor_affine)
  (dropout5): Dropout(p=0, inplace=False)
  (fc3): DynamicQuantizedLinear(in_features=128, out_features=10, qscheme=torch.per_tensor_affine)
)


## Flipping one bit at a time (w/ 0.5 rate). Iterating between which bit to flip to check accuracies

In [16]:
PATH = './sample_models/cifar_net_sgd_72_41.pth'
print(PATH)
print("")

# print(quant_net)  # Can see here that only the linear layers have been quantized

fc_layers = ["fc1._packed_params.weight", "fc2._packed_params.weight", "fc3._packed_params.weight"]

bit_values = [254, 253, 251, 247, 239, 223, 191, 127]


# Load in unmodded quant net for accuracy comparison ----------------------------------------------------------------------------------------------
unmod_quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
unmod_quant_net.load_state_dict(torch.load(PATH))
unmod_quant_net = torch.quantization.quantize_dynamic(unmod_quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)

correct = 0
total = 0 
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = unmod_quant_net(images, False, drop1=0, drop2=0, drop3=0, drop4=0, drop5=0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Unmodded Model Accuracy: %.2f %%' % (100 * correct / float(total)))
print("")


for j in range(len(bit_values)):
    print("Bit value:", j, bit_values[j])
    print("")
    ave_sum = 0
    loops = 10
    
    for k in range(loops):
        
        quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
        quant_net.load_state_dict(torch.load(PATH))
        quant_net = torch.quantization.quantize_dynamic(quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)
    
        for i in range(len(fc_layers)):
        #     print(fc_layers[i])
        #     print("")

            fc_weights = quant_net.state_dict()[fc_layers[i]]
            fc_weights_int = quant_net.state_dict()[fc_layers[i]].int_repr()

            scale = fc_weights.q_scale()
            zero_pt = fc_weights.q_zero_point()

        #     print(fc_weights)
            # print(fc_weights_int)
            # print(fc_weights_int.max(), fc_weights_int.min())

            # ------------------------------------ BIT FLIPPING IS HERE. MODIFY random_arr VALUES AND THRESHOLD TO CHANGE -------------------------

            fc_s = fc_weights.size()
            random_arr = torch.rand(fc_s)

            # print(random_arr)

            # convert based on threshold value (value determines how often bit is flipped from 1-0)
            threshold_v = 0.5
            threshold = 1 - threshold_v

            # BIT MASKING
    #         random_arr[random_arr >= threshold] = 127   # MSB
    #         random_arr[random_arr >= threshold] = 254   # LSB
            random_arr[random_arr >= threshold] = bit_values[j]
            random_arr[random_arr < threshold] = 255
            random_arr_np = random_arr.type(torch.uint8)

            new_fc = fc_weights_int.type(torch.uint8)

            new_arr = torch.bitwise_and(random_arr_np, new_fc).type(torch.int8)

            '''
            # Same stuff as above, just done with numpy arrays here
            random_arr_np = random_arr.type(torch.uint8).numpy()

            # print(random_arr_np)

            new_fc = fc_weights_int.type(torch.uint8).numpy().astype('uint8')
            # print(new_fc)

            new_arr = torch.tensor(np.bitwise_and(random_arr_np, new_fc).astype('int8'))
            # print(new_arr)
            '''


            # --------------------------------Reload new values (or same ones) into the network again --------------------------------

            manual = (new_arr-zero_pt)*scale

            new_quant_tensor = torch.quantize_per_tensor(manual, scale, zero_pt, torch.qint8)

            state_dict = quant_net.state_dict()
            state_dict[fc_layers[i]] = new_quant_tensor
            quant_net.load_state_dict(state_dict)

            c1_w_params = quant_net.state_dict()[fc_layers[i]]

        #     print(c1_w_params)

        #     print("")
        #     print("Sum before:", torch.sum(fc_weights_int).item(), "  Sum after:", torch.sum(new_arr).item())

        #     print("")
        #     print("")
        #     print("")


        # Now check accuracy ---------------------------------------------------------------------------------------------

        correct = 0
        total = 0 
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                outputs = quant_net(images, False, drop1=0, drop2=0, drop3=0, drop4=0, drop5=0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print('Modified Accuracy: %.2f %%' % (100 * correct / float(total)))
    
        ave_sum += 100 * correct / float(total)
        
    print("")
    print('Number of loops:', loops)
    print('Average mod acc is: %.2f %%' % (ave_sum/float(loops)))
    print("")
    print("")

./cifar_net_sgd_72_41.pth

Unmodded Model Accuracy: 72.35 %

Bit value: 0 254

Modified Accuracy: 71.91 %
Modified Accuracy: 71.90 %
Modified Accuracy: 71.87 %
Modified Accuracy: 71.90 %
Modified Accuracy: 71.89 %
Modified Accuracy: 71.88 %
Modified Accuracy: 71.91 %
Modified Accuracy: 71.93 %
Modified Accuracy: 72.01 %
Modified Accuracy: 71.90 %

Number of loops: 10
Average mod acc is: 71.91 %


Bit value: 1 253

Modified Accuracy: 71.30 %
Modified Accuracy: 71.18 %
Modified Accuracy: 71.12 %
Modified Accuracy: 71.16 %
Modified Accuracy: 71.25 %
Modified Accuracy: 71.15 %
Modified Accuracy: 71.09 %
Modified Accuracy: 71.14 %
Modified Accuracy: 71.13 %
Modified Accuracy: 71.23 %

Number of loops: 10
Average mod acc is: 71.17 %


Bit value: 2 251

Modified Accuracy: 69.30 %
Modified Accuracy: 69.28 %
Modified Accuracy: 69.42 %
Modified Accuracy: 69.27 %
Modified Accuracy: 69.10 %
Modified Accuracy: 69.22 %
Modified Accuracy: 69.24 %
Modified Accuracy: 69.25 %
Modified Accuracy: 69.57 %


## Flipping 4 bits at the same time (first 4 or last 4 or none flipped) [00001111 or 11110000 or 11111111 mask used]

In [6]:
# Load in the quantized weights for fc (fc1) layer ----------------------------------------------------------------------------

PATH = './sample_models/cifar_weight_2_1e-3.pth'
print(PATH)
print("")

# print(quant_net)  # Can see here that only the linear layers have been quantized

fc_layers = ["fc1._packed_params.weight", "fc2._packed_params.weight", "fc3._packed_params.weight"]

bit_values = [15, 240]  ## 00001111, 11110000

top_p = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

# Load in unmodded for comparison ----------------------------------------------------------------------------------------------
unmod_quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
unmod_quant_net.load_state_dict(torch.load(PATH))
unmod_quant_net = torch.quantization.quantize_dynamic(unmod_quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)

correct = 0
total = 0 
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = unmod_quant_net(images, False, drop1=0, drop2=0, drop3=0, drop4=0, drop5=0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Unmodded Model Accuracy: %.2f %%' % (100 * correct / float(total)))
print("")



for j in range(len(top_p)):
    print("Top_p", top_p[j], "Bot_p", 1-top_p[j]-0.5)
    print("")
    ave_sum = 0
    loops = 10
    
    for k in range(loops):
        
        quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
        quant_net.load_state_dict(torch.load(PATH))
        quant_net = torch.quantization.quantize_dynamic(quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)
    
        for i in range(len(fc_layers)):
        #     print(fc_layers[i])
        #     print("")

            fc_weights = quant_net.state_dict()[fc_layers[i]]
            fc_weights_int = quant_net.state_dict()[fc_layers[i]].int_repr()

            scale = fc_weights.q_scale()
            zero_pt = fc_weights.q_zero_point()

            # ------------------------------------Insert some bit flipping stuff here ------------------------------------------------
            fc_s = fc_weights.size()
            random_arr = torch.rand(fc_s)

            # convert based on threshold value (value determines how often bit is flipped from 1-0)
            threshold_v = 0.5
            threshold = 1 - threshold_v
            
            top_v = 1 - top_p[j]

            # Bit masking
            random_arr[(random_arr > threshold) & (random_arr > top_v)] = bit_values[0]
            random_arr[(random_arr > threshold) & (random_arr <= top_v)] = bit_values[1]
            random_arr[random_arr <= threshold] = 255
            random_arr_np = random_arr.type(torch.uint8)

            new_fc = fc_weights_int.type(torch.uint8)

            new_arr = torch.bitwise_and(random_arr_np, new_fc).type(torch.int8)


            # --------------------------------Reload new values (or same ones) into the network again --------------------------------

            manual = (new_arr-zero_pt)*scale

            new_quant_tensor = torch.quantize_per_tensor(manual, scale, zero_pt, torch.qint8)

            state_dict = quant_net.state_dict()
            state_dict[fc_layers[i]] = new_quant_tensor
            quant_net.load_state_dict(state_dict)

            c1_w_params = quant_net.state_dict()[fc_layers[i]]


        # Now check accuracy ---------------------------------------------------------------------------------------------

        correct = 0
        total = 0 
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                outputs = quant_net(images, False, drop1=0, drop2=0, drop3=0, drop4=0, drop5=0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print('Modified Accuracy: %.2f %%' % (100 * correct / float(total)))
    
        ave_sum += 100 * correct / float(total)
        
    print("")
    print('Number of loops:', loops)
    print('Average mod acc is: %.2f %%' % (ave_sum/float(loops)))
    print("")
    print("")

./cifar_weight_2_1e-3.pth

Unmodded Model Accuracy: 69.87 %

Top_p 0 Bot_p 0.5

Modified Accuracy: 63.10 %
Modified Accuracy: 63.16 %
Modified Accuracy: 62.18 %
Modified Accuracy: 61.60 %
Modified Accuracy: 61.89 %
Modified Accuracy: 62.50 %
Modified Accuracy: 63.24 %
Modified Accuracy: 62.78 %
Modified Accuracy: 63.01 %
Modified Accuracy: 62.05 %

Number of loops: 10
Average mod acc is: 62.55 %


Top_p 0.1 Bot_p 0.4

Modified Accuracy: 59.69 %
Modified Accuracy: 61.83 %
Modified Accuracy: 60.68 %
Modified Accuracy: 60.45 %
Modified Accuracy: 62.72 %
Modified Accuracy: 61.25 %
Modified Accuracy: 62.12 %
Modified Accuracy: 65.33 %
Modified Accuracy: 60.68 %
Modified Accuracy: 59.27 %

Number of loops: 10
Average mod acc is: 61.40 %


Top_p 0.2 Bot_p 0.30000000000000004

Modified Accuracy: 56.91 %
Modified Accuracy: 55.01 %
Modified Accuracy: 58.31 %
Modified Accuracy: 62.42 %
Modified Accuracy: 57.67 %
Modified Accuracy: 59.54 %
Modified Accuracy: 58.90 %
Modified Accuracy: 61.33 %
Modi

## Only 1 bit can be flipped at a time, 0.5 rate (randomly pick one in upper 4 or lower 4 bits)

In [11]:
# Load in the quantized weights for fc (fc1) layer ----------------------------------------------------------------------------

PATH = './sample_models/cifar_weight_2_1e-3.pth'
print(PATH)
print("")

# print(quant_net)  # Can see here that only the linear layers have been quantized

fc_layers = ["fc1._packed_params.weight", "fc2._packed_params.weight", "fc3._packed_params.weight"]

bit_values_least = [254, 253, 251, 247]
bit_values_most  = [239, 223, 191, 127]

# Load in unmodded for comparison ----------------------------------------------------------------------------------------------
unmod_quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
unmod_quant_net.load_state_dict(torch.load(PATH))
unmod_quant_net = torch.quantization.quantize_dynamic(unmod_quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)

correct = 0
total = 0 
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = unmod_quant_net(images, False, drop1=0, drop2=0, drop3=0, drop4=0, drop5=0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Unmodded Model Accuracy: %.2f %%' % (100 * correct / float(total)))
print("")



for j in range(2):
    if j == 0:
        print("Flipping lower bits")
    elif j == 1:
        print("Flipping upper bits")
    else:
        print("ERROR!!!!!")
        break
    print("")
    ave_sum = 0
    loops = 10
    
    for k in range(loops):
        
        quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
        quant_net.load_state_dict(torch.load(PATH))
        quant_net = torch.quantization.quantize_dynamic(quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)
    
        for i in range(len(fc_layers)):
        #     print(fc_layers[i])
        #     print("")

            fc_weights = quant_net.state_dict()[fc_layers[i]]
            fc_weights_int = quant_net.state_dict()[fc_layers[i]].int_repr()

            scale = fc_weights.q_scale()
            zero_pt = fc_weights.q_zero_point()

            # ------------------------------------Insert some bit flipping stuff here ------------------------------------------------
            fc_s = fc_weights.size()
            random_arr = torch.rand(fc_s)

            # convert based on threshold value (value determines how often bit is flipped from 1-0)
            threshold_v = 0.5
            threshold = 1 - threshold_v
            
            # Manually 1/4 chance for each bit to flip
            t1 = 1 - 0.125
            t2 = 1 - 0.25
            t3 = 1 - 0.375
            t4 = 1 - 0.5
            

            # Bit masking
    #         random_arr[random_arr >= threshold] = 127   # MSB
    #         random_arr[random_arr >= threshold] = 254   # LSB
    
            if j == 0:
                random_arr[random_arr > t1] = bit_values_least[0]
                random_arr[(random_arr > t2) & (random_arr <= t1)] = bit_values_least[1]
                random_arr[(random_arr > t3) & (random_arr <= t2)] = bit_values_least[2]
                random_arr[(random_arr > t4) & (random_arr <= t3)] = bit_values_least[3]
            elif j == 1:
                random_arr[random_arr > t1] = bit_values_most[0]
                random_arr[(random_arr > t2) & (random_arr <= t1)] = bit_values_most[1]
                random_arr[(random_arr > t3) & (random_arr <= t2)] = bit_values_most[2]
                random_arr[(random_arr > t4) & (random_arr <= t3)] = bit_values_most[3]
            
            random_arr[random_arr <= threshold] = 255
            random_arr_np = random_arr.type(torch.uint8)

            new_fc = fc_weights_int.type(torch.uint8)

            new_arr = torch.bitwise_and(random_arr_np, new_fc).type(torch.int8)


            # --------------------------------Reload new values (or same ones) into the network again --------------------------------

            manual = (new_arr-zero_pt)*scale

            new_quant_tensor = torch.quantize_per_tensor(manual, scale, zero_pt, torch.qint8)

            state_dict = quant_net.state_dict()
            state_dict[fc_layers[i]] = new_quant_tensor
            quant_net.load_state_dict(state_dict)

            c1_w_params = quant_net.state_dict()[fc_layers[i]]


        # Now check accuracy ---------------------------------------------------------------------------------------------

        correct = 0
        total = 0 
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                outputs = quant_net(images, False, drop1=0, drop2=0, drop3=0, drop4=0, drop5=0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print('Modified Accuracy: %.2f %%' % (100 * correct / float(total)))
    
        ave_sum += 100 * correct / float(total)
        
    print("")
    print('Number of loops:', loops)
    print('Average mod acc is: %.2f %%' % (ave_sum/float(loops)))
    print("")
    print("")

./cifar_weight_2_1e-3.pth

Unmodded Model Accuracy: 69.87 %

Flipping lower bits

Modified Accuracy: 68.60 %
Modified Accuracy: 68.94 %
Modified Accuracy: 69.29 %
Modified Accuracy: 68.83 %
Modified Accuracy: 68.80 %
Modified Accuracy: 69.32 %
Modified Accuracy: 69.37 %
Modified Accuracy: 68.75 %
Modified Accuracy: 69.23 %
Modified Accuracy: 68.64 %

Number of loops: 10
Average mod acc is: 68.98 %


Flipping upper bits

Modified Accuracy: 26.14 %
Modified Accuracy: 19.04 %
Modified Accuracy: 19.18 %
Modified Accuracy: 23.95 %
Modified Accuracy: 29.09 %
Modified Accuracy: 19.32 %
Modified Accuracy: 28.46 %
Modified Accuracy: 19.58 %
Modified Accuracy: 20.93 %
Modified Accuracy: 22.41 %

Number of loops: 10
Average mod acc is: 22.81 %




## Flipping 1 random bit in upper and lower at same time at different rates

In [13]:
# Load in the quantized weights for fc (fc1) layer ----------------------------------------------------------------------------

PATH = './sample_models/cifar_weight_2_1e-3.pth'
print(PATH)
print("")

# print(quant_net)  # Can see here that only the linear layers have been quantized

fc_layers = ["fc1._packed_params.weight", "fc2._packed_params.weight", "fc3._packed_params.weight"]

bit_values_least = [254, 253, 251, 247]
bit_values_most  = [239, 223, 191, 127]

top_p = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

# Load in unmodded for comparison ----------------------------------------------------------------------------------------------
unmod_quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
unmod_quant_net.load_state_dict(torch.load(PATH))
unmod_quant_net = torch.quantization.quantize_dynamic(unmod_quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)

correct = 0
total = 0 
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = unmod_quant_net(images, False, drop1=0, drop2=0, drop3=0, drop4=0, drop5=0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Unmodded Model Accuracy: %.2f %%' % (100 * correct / float(total)))
print("")



for j in range(len(top_p)):
    print("Top_p", top_p[j], "Bot_p", 1-top_p[j]-0.5)
    print("")
    ave_sum = 0
    loops = 10
    
    for k in range(loops):
        
        quant_net = Net(train_drop1=0, train_drop2=0, train_drop3=0, train_drop4=0, train_drop5=0)
        quant_net.load_state_dict(torch.load(PATH))
        quant_net = torch.quantization.quantize_dynamic(quant_net, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)
    
        for i in range(len(fc_layers)):
        #     print(fc_layers[i])
        #     print("")

            fc_weights = quant_net.state_dict()[fc_layers[i]]
            fc_weights_int = quant_net.state_dict()[fc_layers[i]].int_repr()

            scale = fc_weights.q_scale()
            zero_pt = fc_weights.q_zero_point()

            # ------------------------------------Insert some bit flipping stuff here ------------------------------------------------
            fc_s = fc_weights.size()
            random_arr = torch.rand(fc_s)

            # convert based on threshold value (value determines how often bit is flipped from 1-0)
            threshold_v = 0.5
            threshold = 1 - threshold_v
            
            top_flip = top_p[j]
            bot_flip = 1 - top_flip - threshold
            
            t1 = 1 - top_flip/4
            t2 = 1 - 2*top_flip/4
            t3 = 1 - 3*top_flip/4
            t4 = 1 - 4*top_flip/4
            
            t1_2 = t4 - bot_flip/4
            t2_2 = t4 - 2*bot_flip/4
            t3_2 = t4 - 3*bot_flip/4
            t4_2 = t4 - 4*bot_flip/4
            
            random_arr[random_arr > t1] = bit_values_most[0]
            random_arr[(random_arr > t2) & (random_arr <= t1)] = bit_values_most[1]
            random_arr[(random_arr > t3) & (random_arr <= t2)] = bit_values_most[2]
            random_arr[(random_arr > t4) & (random_arr <= t3)] = bit_values_most[3]
            
            random_arr[(random_arr > t1_2) & (random_arr <= t4)] = bit_values_least[0]
            random_arr[(random_arr > t2_2) & (random_arr <= t1_2)] = bit_values_least[1]
            random_arr[(random_arr > t3_2) & (random_arr <= t2_2)] = bit_values_least[2]
            random_arr[(random_arr > t4_2) & (random_arr <= t3_2)] = bit_values_least[3]
            
            random_arr[random_arr <= threshold] = 255
            random_arr_np = random_arr.type(torch.uint8)

            new_fc = fc_weights_int.type(torch.uint8)

            new_arr = torch.bitwise_and(random_arr_np, new_fc).type(torch.int8)


            # --------------------------------Reload new values (or same ones) into the network again --------------------------------

            manual = (new_arr-zero_pt)*scale

            new_quant_tensor = torch.quantize_per_tensor(manual, scale, zero_pt, torch.qint8)

            state_dict = quant_net.state_dict()
            state_dict[fc_layers[i]] = new_quant_tensor
            quant_net.load_state_dict(state_dict)

            c1_w_params = quant_net.state_dict()[fc_layers[i]]


        # Now check accuracy ---------------------------------------------------------------------------------------------

        correct = 0
        total = 0 
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                outputs = quant_net(images, False, drop1=0, drop2=0, drop3=0, drop4=0, drop5=0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print('Modified Accuracy: %.2f %%' % (100 * correct / float(total)))
    
        ave_sum += 100 * correct / float(total)
        
    print("")
    print('Number of loops:', loops)
    print('Average mod acc is: %.2f %%' % (ave_sum/float(loops)))
    print("")
    print("")

./cifar_weight_2_1e-3.pth

Unmodded Model Accuracy: 69.87 %

Top_p 0 Bot_p 0.5

Modified Accuracy: 69.42 %
Modified Accuracy: 69.36 %
Modified Accuracy: 68.63 %
Modified Accuracy: 68.91 %
Modified Accuracy: 69.13 %
Modified Accuracy: 68.76 %
Modified Accuracy: 69.22 %
Modified Accuracy: 69.18 %
Modified Accuracy: 69.44 %
Modified Accuracy: 69.17 %

Number of loops: 10
Average mod acc is: 69.12 %


Top_p 0.1 Bot_p 0.4

Modified Accuracy: 64.66 %
Modified Accuracy: 57.92 %
Modified Accuracy: 55.22 %
Modified Accuracy: 64.95 %
Modified Accuracy: 63.27 %
Modified Accuracy: 61.56 %
Modified Accuracy: 62.50 %
Modified Accuracy: 65.21 %
Modified Accuracy: 59.78 %
Modified Accuracy: 61.16 %

Number of loops: 10
Average mod acc is: 61.62 %


Top_p 0.2 Bot_p 0.30000000000000004

Modified Accuracy: 49.38 %
Modified Accuracy: 45.30 %
Modified Accuracy: 54.03 %
Modified Accuracy: 54.48 %
Modified Accuracy: 52.58 %
Modified Accuracy: 48.54 %
Modified Accuracy: 46.86 %
Modified Accuracy: 37.92 %
Modi